In [ ]:
import os
import torchvision
import torchvision.transforms as transforms

if not os.path.exists("../data"):
    os.makedirs("../data")

cifar10_train = torchvision.datasets.CIFAR10(
    root="../data",
    train=True,
    download=True,
    transform=transforms.ToTensor()
)
cifar10_test = torchvision.datasets.CIFAR10(
    root="../data",
    train=False,
    download=True,
    transform=transforms.ToTensor()
)
print("✅ CIFAR10 disponible en carpeta '../data/'")

# Análisis Comparativo Final - CNNs Clásicas

Este notebook genera las tablas, figuras y análisis requeridos por la metodología IMRA:

## Contenido
1. **Tabla 1**: Desempeño y costo de modelos base
2. **Tabla 2**: Variantes por modelo y su impacto
3. **Figura 1**: Curvas de pérdida (entrenamiento/validación)
4. **Figura 2**: Curvas de exactitud (validación)
5. **Figura 3**: Matriz de confusión del mejor modelo
6. **Análisis**: Discusión de resultados y conclusiones

In [ ]:
import sys
sys.path.append('../')

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pickle
import os

from utils.visualization import plot_model_comparison, plot_combined_training_curves

# Configurar estilo de visualización
plt.style.use('default')
sns.set_palette("husl")

print("Cargando resultados de todos los experimentos...")

In [ ]:
# Cargar resultados de cada modelo
df_lenet = pd.read_csv('../results/lenet_results.csv')
df_alexnet = pd.read_csv('../results/alexnet_results.csv')
df_vgg16 = pd.read_csv('../results/vgg16_results.csv')

# Cargar historiales de entrenamiento
with open('../results/lenet_histories.pkl', 'rb') as f:
    lenet_histories = pickle.load(f)
with open('../results/alexnet_histories.pkl', 'rb') as f:
    alexnet_histories = pickle.load(f)
with open('../results/vgg16_histories.pkl', 'rb') as f:
    vgg16_histories = pickle.load(f)

print("Datos cargados exitosamente")

## Tabla 1: Desempeño del modelo base

In [ ]:
# Crear Tabla 1: Modelos base
tabla1_data = {
    'Modelo': ['LeNet', 'AlexNet', 'VGG16'],
    'Params (M)': [
        df_lenet[df_lenet['Modelo'] == 'LeNet Base']['Params (M)'].iloc[0],
        df_alexnet[df_alexnet['Modelo'] == 'AlexNet Base']['Params (M)'].iloc[0],
        df_vgg16[df_vgg16['Modelo'] == 'VGG16 Base']['Params (M)'].iloc[0]
    ],
    't/epoca (s)': [
        df_lenet[df_lenet['Modelo'] == 'LeNet Base']['t/epoca (s)'].iloc[0],
        df_alexnet[df_alexnet['Modelo'] == 'AlexNet Base']['t/epoca (s)'].iloc[0],
        df_vgg16[df_vgg16['Modelo'] == 'VGG16 Base']['t/epoca (s)'].iloc[0]
    ],
    'Val Acc': [
        df_lenet[df_lenet['Modelo'] == 'LeNet Base']['Val Acc'].iloc[0],
        df_alexnet[df_alexnet['Modelo'] == 'AlexNet Base']['Val Acc'].iloc[0],
        df_vgg16[df_vgg16['Modelo'] == 'VGG16 Base']['Val Acc'].iloc[0]
    ],
    'Val F1': [
        df_lenet[df_lenet['Modelo'] == 'LeNet Base']['Val F1'].iloc[0],
        df_alexnet[df_alexnet['Modelo'] == 'AlexNet Base']['Val F1'].iloc[0],
        df_vgg16[df_vgg16['Modelo'] == 'VGG16 Base']['Val F1'].iloc[0]
    ],
    'Test Acc': [
        df_lenet[df_lenet['Modelo'] == 'LeNet Base']['Test Acc'].iloc[0],
        df_alexnet[df_alexnet['Modelo'] == 'AlexNet Base']['Test Acc'].iloc[0],
        df_vgg16[df_vgg16['Modelo'] == 'VGG16 Base']['Test Acc'].iloc[0]
    ],
    'Test F1': [
        df_lenet[df_lenet['Modelo'] == 'LeNet Base']['Test F1'].iloc[0],
        df_alexnet[df_alexnet['Modelo'] == 'AlexNet Base']['Test F1'].iloc[0],
        df_vgg16[df_vgg16['Modelo'] == 'VGG16 Base']['Test F1'].iloc[0]
    ]
}

tabla1 = pd.DataFrame(tabla1_data)
print("TABLA 1: Desempeño del modelo base")
print("="*50)
print(tabla1.round(2))
tabla1.to_csv('../results/tabla1_modelos_base.csv', index=False)

## Tabla 2: Variantes por modelo

In [ ]:
# Crear Tabla 2: Variantes
def calcular_cambios(df_modelo, modelo_base):
    base_acc = df_modelo[df_modelo['Modelo'] == modelo_base]['Test Acc'].iloc[0]
    base_f1 = df_modelo[df_modelo['Modelo'] == modelo_base]['Test F1'].iloc[0]
    
    variantes = []
    for _, row in df_modelo.iterrows():
        if row['Modelo'] != modelo_base:
            cambio_acc = row['Test Acc'] - base_acc
            cambio_f1 = row['Test F1'] - base_f1
            variantes.append({
                'Modelo': row['Modelo'].split(' ', 1)[0],
                'Variante': row['Modelo'].split(' ', 1)[1] if ' ' in row['Modelo'] else 'Base',
                'Cambio clave': get_cambio_descripcion(row['Modelo']),
                'Cambio Accuracy': cambio_acc,
                'Cambio F1': cambio_f1,
                'Observaciones': get_observaciones(cambio_acc, cambio_f1)
            })
    return variantes

def get_cambio_descripcion(modelo):
    if 'BN' in modelo or 'BatchNorm' in modelo:
        return 'BatchNorm tras conv'
    elif 'Enhanced' in modelo:
        return 'Incremento canales'
    elif 'Reduced' in modelo:
        return 'Reducción FC'
    elif 'GAP' in modelo:
        return 'FC -> GAP'
    return 'Modificación'

def get_observaciones(cambio_acc, cambio_f1):
    if cambio_acc > 2 and cambio_f1 > 2:
        return 'Mejora significativa'
    elif cambio_acc > 0 and cambio_f1 > 0:
        return 'Mejora moderada'
    elif cambio_acc < -2 or cambio_f1 < -2:
        return 'Degradación'
    else:
        return 'Sin cambio significativo'

# Calcular cambios
variantes_lenet = calcular_cambios(df_lenet, 'LeNet Base')
variantes_alexnet = calcular_cambios(df_alexnet, 'AlexNet Base')
variantes_vgg16 = calcular_cambios(df_vgg16, 'VGG16 Base')

todas_variantes = variantes_lenet + variantes_alexnet + variantes_vgg16
tabla2 = pd.DataFrame(todas_variantes)

print("TABLA 2: Variantes por modelo")
print("="*50)
print(tabla2.round(2))
tabla2.to_csv('../results/tabla2_variantes.csv', index=False)

## Figuras de Análisis

In [ ]:
# Figura 1 y 2: Curvas de pérdida y exactitud
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))

modelos_base = {
    'LeNet': lenet_histories['base'],
    'AlexNet': alexnet_histories['base'],
    'VGG16': vgg16_histories['base']
}

for nombre, historia in modelos_base.items():
    epochs = range(1, len(historia['val_loss'])+1)
    ax1.plot(epochs, historia['val_loss'], label=nombre)
    ax2.plot(epochs, historia['val_acc'], label=nombre)

ax1.set_title('Figura 1: Curvas de Pérdida (Validación)')
ax1.set_xlabel('Época')
ax1.set_ylabel('Loss')
ax1.legend()

ax2.set_title('Figura 2: Curvas de Exactitud (Validación)')
ax2.set_xlabel('Época')
ax2.set_ylabel('Accuracy (%)')
ax2.legend()

plt.tight_layout()
plt.savefig('../results/figura1_2_curvas_combinadas.png')
plt.show()

In [ ]:
# Figura 3 y 4: Comparación de rendimiento y eficiencia
plot_model_comparison(tabla1, '../results/figura3_comparacion_rendimiento.png')

fig, ax = plt.subplots(figsize=(8,6))
ax.scatter(tabla1['Params (M)'], tabla1['Test Acc'], s=tabla1['t/epoca (s)']*10)
for i, model in enumerate(tabla1['Modelo']):
    ax.annotate(model, (tabla1['Params (M)'].iloc[i], tabla1['Test Acc'].iloc[i]))
ax.set_xlabel('Parámetros (M)')
ax.set_ylabel('Test Acc (%)')
ax.set_title('Eficiencia: Rendimiento vs Complejidad')
plt.savefig('../results/figura4_eficiencia.png')
plt.show()